In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk, sep


from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.layouts import row, column
from bokeh.io import save, export_png
from bokeh.palettes import Dark2_5 as palette
from IPython.display import display, Markdown
import itertools
output_notebook()

Loading BokehJS ...

In [56]:
SOURCE_DIR = 'results/results_net_1_B_MDC_registered'

In [57]:
dfs = {}
for (dirpath, dirname, filenames) in walk(SOURCE_DIR):
    for filename in filenames:
        if filename == "metrics.csv":
            name = dirpath.split(sep)[-1]
            dfs[name] = pd.read_csv(sep.join([dirpath, filename]))

for key, df in dfs.items():
    print(f'---{key}---')
    print(df.describe())
    print()

---P25---
       Unnamed: 0        DCI         TI         Em        Bpn
count   85.000000  85.000000  85.000000  85.000000  85.000000
mean    42.000000   0.984820   0.937436   0.062564  -0.013967
std     24.681302   0.004801   0.021253   0.021253   0.023529
min      0.000000   0.970823   0.873967   0.028006  -0.090623
25%     21.000000   0.981824   0.927214   0.049580  -0.025250
50%     42.000000   0.984774   0.942351   0.057649  -0.012112
75%     63.000000   0.989636   0.950420   0.072786   0.002338
max     84.000000   0.992427   0.971994   0.126033   0.041681

---P18---
       Unnamed: 0        DCI         TI         Em        Bpn
count    72.00000  72.000000  72.000000  72.000000  72.000000
mean     35.50000   0.974472   0.840765   0.159235  -0.019416
std      20.92845   0.004562   0.037072   0.037072   0.033410
min       0.00000   0.968170   0.734064   0.089235  -0.086245
25%      17.75000   0.970857   0.824395   0.136576  -0.042089
50%      35.50000   0.973602   0.845897   0.15410

In [58]:
for name, df in dfs.items():
    t = df.iloc[:,1:]
    colors = itertools.cycle(palette) 
    p = figure(x_range=t['Filename'])
    p.title = name
    x = [x for x in range(len(t['Filename'])) ]

    for feat, color in zip(t.iloc[:, 1:], colors):
        p.line(t['Filename'], t[feat], line_width=2, color=color, legend_label=f"{feat} {name}")
        p.legend.location = "top_left"
        p.legend.click_policy="mute"
        p.xaxis.major_label_orientation = "vertical"
    show(p)

In [59]:
def calculate_data_for_plots(metric: str) -> dict[str, float]:
    tot_res = {}
    indici = list(dfs['P18'].describe().keys()[1:])
    for indice in indici:
        res = []
        for name, df in dfs.items():
            tmp = df.describe().transpose()[metric].transpose()
            res.append((name,tmp[indice]))
        res = [x[1] for x in sorted(res, key=lambda l: l[0])]
        tot_res[indice] = res
    return tot_res

In [60]:
statistiche = list(dfs['P18'].describe().transpose().keys())
for statistica in statistiche:
       #t = dfs['P18'].describe().transpose()['mean'].transpose()
       names_ordered = sorted(dfs.keys())
       act_values = calculate_data_for_plots(statistica)
       data = {'x' : names_ordered,
              'DCI'   : act_values['DCI'],
              'TI'   : act_values['TI'],
              'Em'   : act_values['Em'],
              'Bpn': act_values['Bpn'],}

       source = ColumnDataSource(data=data)
       p = figure(x_range=names_ordered, title=statistica.upper(),
              height=350)


       p.vbar(x=dodge('x', -0.37, range=p.x_range), top='DCI', source=source,
              width=0.2, color="#c9d9d3", legend_label="DCI")

       p.vbar(x=dodge('x',  -0.12,  range=p.x_range), top='TI', source=source,
              width=0.2, color="#718dbf", legend_label="TI")

       p.vbar(x=dodge('x',  0.12, range=p.x_range), top='Em', source=source,
              width=0.2, color="#e84d60", legend_label="Em")

       p.vbar(x=dodge('x',  0.37, range=p.x_range), top='Bpn', source=source,
              width=0.2, color="#6f7c00", legend_label="Bpn")

       export_png(p, filename=f"Statistics_graphs/{statistica}.png")
       show(p)

In [61]:
for name, df in dfs.items():
    t = df.iloc[:,1:]
    colors = itertools.cycle(palette) 
    p = figure(x_range=t['Filename'])
    p.title = name
    x = [x for x in range(len(t['Filename'])) ]

    for feat, color in zip(t.iloc[:, 1:], colors):
        p.line(t['Filename'], t[feat], line_width=2, color=color, legend_label=f"{feat} {name}")
        p.legend.location = "top_left"
        p.legend.click_policy="mute"
        p.xaxis.major_label_orientation = "vertical"
    export_png(p, filename=f"Statistics_graphs/{name}.png")
    show(p)